In [10]:
# confident edges: edges that iterative approach predicted early (in other words, "confidently")
# we compare our method and LR method on the accuracy of those edges


import _pickle as pkl
import networkx as nx
import numpy as np
import random

from tqdm import tqdm
from snpp.cores.lowrank import alq_spark, predict_signs
from snpp.utils.matrix import split_train_test, load_sparse_csr
from snpp.utils.signed_graph import g2m
from snpp.utils.data import load_train_test_graphs
from snpp.utils.edge_filter import filter_by_min_triangle_count

from snpp.utils.spark import sc

dataset = 'slashdot'
lambda_ = 0.2
k = 10
max_iter = 100
random_seed = 123456
min_tri_count = 25

recache_input = False

random.seed(random_seed)
np.random.seed(random_seed)

In [2]:
train_g, test_g = load_train_test_graphs(dataset, recache_input)


loading train and test graphs...


In [3]:
################
## DEBUG
#################
print(train_g.number_of_nodes())
print(train_g.number_of_edges())
print(test_g.number_of_edges())
assert train_g.number_of_nodes() == test_g.number_of_nodes()

77357
464917
51658


In [4]:
train_g_ud = train_g.to_undirected()

train_m = g2m(train_g)
train_m_ud = g2m(train_g_ud)
print(train_m_ud.shape)
print(train_m_ud.nnz)

100%|██████████| 77357/77357 [00:01<00:00, 58879.16it/s]


(77357, 77357)
929435


In [11]:
confident_edges = set(filter_by_min_triangle_count(train_g_ud, test_g.edges(), min_tri_count))
print(len(confident_edges))

2142


In [12]:
X, Y = alq_spark(train_m, k=k, sc=sc,
                 lambda_=lambda_, iterations=max_iter,
                 seed=random_seed)

KeyboardInterrupt: 

In [13]:
truth = set((i, j, test_g[i][j]['sign']) for i, j in confident_edges)
preds = predict_signs(X, Y, confident_edges, sc)
assert len(truth) == len(preds)
assert set((i, j) for i, j, _ in preds) == set((i ,j) for i, j, _ in truth)

print('=> accuracy on {} edges {} (lowrank)'.format(len(preds), len(truth.intersection(preds)) / len(truth)))

=> accuracy on 2142 edges 0.9822595704948646 (lowrank)


In [ ]:
## Iterative approach
from snpp.cores.joint_part_pred import iterative_approach
from snpp.cores.max_balance import faster_greedy
from snpp.cores.lowrank import partition_graph
from snpp.cores.budget_allocation import constant_budget
from snpp.cores.triangle import build_edge2edges

part, iter_preds, status = iterative_approach(
    train_g_ud,
    T=confident_edges,
    k=k,
    graph_partition_f=partition_graph,
    graph_partition_kwargs=dict(sc=sc,
                                lambda_=lambda_,
                                iterations=max_iter,
                                seed=random_seed),
    budget_allocation_f=constant_budget,
    budget_allocation_kwargs=dict(const=200),
    solve_maxbalance_f=faster_greedy,
    solve_maxbalance_kwargs={'edge2edges': build_edge2edges(train_g_ud.copy(),
                                                            confident_edges)},
    truth=set([(i, j, test_g[i][j]['sign'])
               for i, j in confident_edges]),
    perform_last_partition=False)


 19%|█▊        | 399/2142 [00:00<00:00, 2007.60it/s]

build edge2edges


100%|██████████| 2142/2142 [00:01<00:00, 1987.73it/s]


iteration=1, #remaining targets=2142
graph partitioning...
to_scipy_sparse_matrix


100%|██████████| 77357/77357 [00:01<00:00, 59590.46it/s]


ALS...


In [9]:
## Confusion matrix
from sklearn.metrics import confusion_matrix

true_signs = [s for _, _, s in sorted(list(truth))] 
pred_signs = [s for _, _, s in sorted(list(preds))] 
print(confusion_matrix(true_signs, pred_signs, labels=[-1, 1]))

[[ 149   46]
 [  16 2434]]


In [ ]:
# Conclusion:
# Strong balance is more reliable than weak balance
# however, is it because of the poor quality of partitioning?